# Homework 6-3

---

## Imports

In [1]:
from thermostate import State, Q_, units
import numpy as np
from numpy import sqrt, log

---

## Definitions

In [2]:
substance = 'air'
m = Q_(1.0, 'kg')
V_1 = Q_(0.01, 'm**3')
p_1 = Q_(70.0, 'bar')
V_2 = Q_(0.05, 'm**3')

---

## Problem Statement

1.0 kg of air initially occupying a volume of 0.01 m<sup>3</sup> at a pressure of 70.0 bar expands in a piston-cylinder assembly isothermally and without irreversibilities until the volume is 0.05 m<sup>3</sup>. Using the Redlich-Kwong equation of state, determine the

1. temperature during the process, in K
2. final pressure, in bar
3. work developed in the process, in kJ

Hint: The equation for the work in a piston-cylinder is
$$W = n \int_{\bar{v}_1}^{\bar{v}_2} p d \bar{v}$$
You will need to perform the symbolic integration of the R-K EOS and then plug in numbers

---

## Solution

### 1. the temperature in the process

The R-K EOS is
$$p = \frac{\overline{R} T}{\overline{v} - b} - \frac{a}{\overline{v}(\overline{v} + b)\sqrt{T}}$$

This equation cannot be solved for the temperature explicitly, so a guess-and-check method must be used. We guess a temperature, then check the pressure is equal to what we expect.

In [3]:
MW_air = Q_(28.97, 'g/mol')
v_bar_1 = V_1*MW_air/m
a = Q_(15.989, 'bar*(m**6/kmol**2)*K**(1/2)')
b = Q_(0.02541, 'm**3/kmol')
R_bar = units.molar_gas_constant

# Use a bisection method to find the temperature
err = 1.0
T_guess_a = Q_(200.0, 'K')
p_guess_a = R_bar*T_guess_a/(v_bar_1 - b) - a/(v_bar_1*(v_bar_1 + b)*np.sqrt(T_guess_a))
err_a = (p_guess_a - p_1)/p_1
T_guess_b = Q_(300.0, 'K')
while abs(err) >= 1.0E-6:
    T_guess = (T_guess_a + T_guess_b)/2
    p_guess = R_bar*T_guess/(v_bar_1 - b) - a/(v_bar_1*(v_bar_1 + b)*np.sqrt(T_guess))
    err = (p_guess - p_1)/p_1
    if np.sign(err) == np.sign(err_a):
        T_guess_a = T_guess
        p_guess_a = R_bar*T_guess_a/(v_bar_1 - b) - a/(v_bar_1*(v_bar_1 + b)*np.sqrt(T_guess_a))
        err_a = (p_guess_a - p_1)/p_1
    else:
        T_guess_b = T_guess

T = T_guess

<div class="alert alert-success">

**Answer:** The temperature during this process is $T =$ 257.22 K

</div>

### 2. the final pressure

Now that we know the temperature, we can apply the R-K EOS to find the final pressure.

In [4]:
v_bar_2 = V_2*MW_air/m
p_2 = (units.molar_gas_constant*T/(v_bar_2 - b) - a/(v_bar_2*(v_bar_2 + b)*np.sqrt(T))).to('bar')

<div class="alert alert-success">

**Answer:** The final presure is $p_2 =$ 14.56 bar

</div>

### 3. the work output

To find the work output, we have to integrate the R-K EOS as a function of the molar specific volume
\begin{align*}
W &= n\int_{\overline{v}_1}^{\overline{v}_2} p d \overline{v} \\
W &= n\int_{\overline{v}_1}^{\overline{v}_2} \left[\frac{\overline{R} T}{\overline{v} - b} - \frac{a}{\overline{v}(\overline{v} + b)\sqrt{T}}\right] d \overline{v} \\
W &= n\left\{\overline{R}T\ln{\left[\frac{b - \overline{v}_2}{b - \overline{v}_1}\right]} - \frac{a \ln{\left[\frac{\left(b + \overline{v}_1\right)\overline{v}_2}{\left(b + \overline{v}_2\right)\overline{v}_1}\right]}}{b\sqrt{T}}\right\}
\end{align*}

In [5]:
n = m/MW_air
first_log = (b - v_bar_2)/(b - v_bar_1)
second_log = ((b + v_bar_1)*v_bar_2)/((b + v_bar_2)*v_bar_1)
W = (n*(units.molar_gas_constant*T*np.log(first_log) - (a*np.log(second_log)/(b*np.sqrt(T))))).to('kJ')

<div class="alert alert-success">

**Answer:** The work is $W =$ 115.25 kJ

</div>